*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#extract data from the respective tsv files. 

train_data = pd.read_csv(train_file_path,sep='\t')

test_data = pd.read_csv(test_file_path,sep='\t')

In [ ]:
#separate data into messages and labels.

#messages from the dataset.
train_messages = train_data['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.']
test_messages = test_data['i am in hospital da. . i will return home in evening']

#labels from the data set.
train_labels = train_data['ham']
test_labels =  test_data['ham']

In [ ]:
#https://stackoverflow.com/questions/57030670/how-to-remove-punctuation-and-numbers-during-tweettokenizer-step-in-nlp
import regex as re
import string
def clean_text(text):
    #remove html tags
    p = re.compile(r'<[\w+#]>')
    p= p.sub('', text)
    # remove numbers
    text_nonum = re.sub(r'\d+', '', p)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    #remove websites
    remove_urls = re.sub('http[s]?://\S+', '', text_nopunct)
    # substitute multiple whitespace with single whitespace
    return remove_urls


In [ ]:
#clean and remove uneeded chars from sentences
cleaned_train = [clean_text(sent) for sent in train_messages]
cleaned_test = [clean_text(sent) for sent in test_messages]

In [ ]:
#using the nnlm-en-dim50nnlm-en-dim50 layer
import tensorflow_hub as hub
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(cleaned_train)

In [ ]:
#vectorize labels
dict_labels = {'ham':0,'spam':1}
labels_train = [dict_labels[label] for label in train_labels]
labels_test = [dict_labels[label] for label in test_labels]

In [ ]:
#ref tensorflow docs
#build model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#convert data to tensors
train_messages = tf.convert_to_tensor(train_messages)
test_messages = tf.convert_to_tensor(test_messages)

In [ ]:
#convert labels to tensors
train_labels = tf.convert_to_tensor(labels_train)
test_labels = tf.convert_to_tensor(labels_test)

In [ ]:
#fit model to data
history = model.fit(train_messages,train_labels,batch_size=100,epochs=10,verbose=1,validation_data=(train_messages,train_labels),) 

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = [pred_text]
  prediction = model.predict(pred)[0][0]
  label = ''
  if prediction > 0.002:
    label = 'spam'

  else:
    label = 'ham'

  return ([prediction,label])


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
